In [1]:
%run clean_data_en_lstm
%run clean_features

from collections import defaultdict, namedtuple
import numpy as np
import pandas as pd
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
#import duolingo_replica_alex as dr
#import duolingo_replica as d
import dask.dataframe as dd




time_steps_range = range(20, 201, 20)  # Time steps from 20 to 200 in steps of 20
mae_results = {}

for time_steps in time_steps_range:
  
    print(f"Training model with time_steps: {time_steps}")
        
    # time_steps = 20

    file_path = "https://www.dropbox.com/scl/fi/pnxa2jv4xf23bfwry1q9x/learning_traces.13m.csv?rlkey=2dt9848lutbgyys5sujq8dgw2&dl=1"
    result_df_en_top20 = process_data(file_path, time_steps)

    # Clean features 
    clean_data, feature_vars = read_data( result_df_en_top20, omit_lexemes = False )

    # Create time features 
    feature_vars_time = feature_vars + ['time_afternoon', 'time_evening', 'time_morning', 'time_night']

    import numpy as np
    from sklearn.model_selection import train_test_split

    # Assuming clean_data is your DataFrame, Y is the target column, and feature_vars are the features
    # Extract unique users
    unique_users = clean_data['user_id'].unique()  # Replace 'user_id' with your user identifier column

    # Split users into train and test groups
    train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)  # Adjust test_size as needed

    # Create training and testing datasets
    train_data = clean_data[clean_data['user_id'].isin(train_users)]
    test_data = clean_data[clean_data['user_id'].isin(test_users)]

    # Pre-sort the data by 'user_id' and 'datetime'
    train_data.sort_values(by=['user_id', 'datetime'], inplace= True)
    test_data.sort_values(by=['user_id', 'datetime'], inplace= True)

    # Create Batches so we cann run the model
    import numpy as np

    def create_non_overlapping_sequences(data, feature_vars, time_steps=time_steps, batch_size=1000):
        # Convert feature columns to numpy for faster processing
        feature_data = data[feature_vars].to_numpy()
        output_data = data['p'].to_numpy()  # Assuming 'p' is your target variable

        # Get the starting index for each new user
        user_change_indices = np.where(data['user_id'].to_numpy()[:-1] != data['user_id'].to_numpy()[1:])[0] + 1
        user_start_indices = np.insert(user_change_indices, 0, 0)

        for batch_start in range(0, len(user_start_indices), batch_size):
            sequences = []
            outputs = []

            # Iterate through each user in the batch
            for i in range(batch_start, min(batch_start + batch_size, len(user_start_indices))):
                start_idx = user_start_indices[i]
                end_idx = start_idx + time_steps if i + 1 < len(user_start_indices) else len(feature_data)

                # Check if the user data is exactly equal to time_steps
                if end_idx - start_idx == time_steps:
                    sequences.append(feature_data[start_idx:end_idx])
                    outputs.append(output_data[end_idx - 1])  # Target for the last period in the sequence

            yield np.array(sequences), np.array(outputs)

    # Complex model 
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.callbacks import EarlyStopping
    from tensorflow.keras.regularizers import L1L2

    # # Enable mixed precision training
    # from tensorflow.keras.mixed_precision import experimental as mixed_precision
    # policy = mixed_precision.Policy('mixed_float16')
    # mixed_precision.set_policy(policy)

    # # Create and compile the model within a strategy scope if using multiple GPUs
    # strategy = tf.distribute.MirroredStrategy()

    # with strategy.scope():
    model = Sequential()
    model.add(LSTM(100, input_shape=(time_steps, len(feature_vars_time)), return_sequences=True, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(LSTM(50, activation='relu', return_sequences=False))
    model.add(Dense(1, kernel_regularizer=L1L2(l1=0.01, l2=0.01)))
    # Use experimental options for potential speed-up
    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    from tensorflow.keras.metrics import MeanAbsoluteError
    import numpy as np

    # Training model 
    # Initialize the MAE metric
    mae_metric = MeanAbsoluteError()
    num_epochs = 20
    # Train the model using the generator
    for epoch in range(num_epochs):  # You can define the number of epochs
        print("Epoch {}/{}".format(epoch + 1, num_epochs))
        for X_batch, Y_batch in create_non_overlapping_sequences(train_data, feature_vars_time):
            # Train on batch and compute loss
            loss = model.train_on_batch(X_batch, Y_batch)

            # Compute MAE
            predictions = model.predict_on_batch(X_batch)
            mae_metric.update_state(Y_batch, predictions)
            mae = mae_metric.result().numpy()  # Get the MAE value

            # Reset MAE metric at the end of each batch
            mae_metric.reset_states()

            # Optionally print the loss and MAE for each batch
            print("Batch Loss: {:.4f}, Batch MAE: {:.4f}".format(loss, mae))

    # Initialize metric for testing
    test_mae_metric = MeanAbsoluteError()

    # Run model on test data
    for X_test_batch, Y_test_batch in create_non_overlapping_sequences(test_data, feature_vars_time):
        # Make predictions
        test_predictions = model.predict_on_batch(X_test_batch)

        # Update MAE metric
        test_mae_metric.update_state(Y_test_batch, test_predictions)
        print(test_mae_metric.result())

    # Calculate final MAE on test data
    final_test_mae = test_mae_metric.result().numpy()
    print("Final Test MAE: {:.6f}".format(final_test_mae))

    # Save the result
    mae_results[time_steps] = final_test_mae

# Save MAE results to a text file
with open('mae_results.txt', 'w') as file:
    for time_steps, mae in mae_results.items():
        file.write(f"{time_steps}: {mae}\n")

print("MAE results saved to mae_results.txt")


Training model with time_steps: 40


c:\Users\Alexander\anaconda3\envs\tensorflow\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Epoch 1/20
Batch Loss: 1.0560, Batch MAE: 0.8058
Batch Loss: 0.8382, Batch MAE: 0.7220
Batch Loss: 0.6453, Batch MAE: 0.6002
Batch Loss: 0.4879, Batch MAE: 0.4627
Batch Loss: 0.3365, Batch MAE: 0.3129
Batch Loss: 0.2254, Batch MAE: 0.2433
Batch Loss: 0.2650, Batch MAE: 0.2950
Batch Loss: 0.3029, Batch MAE: 0.2541
Batch Loss: 0.2464, Batch MAE: 0.2079
Batch Loss: 0.2141, Batch MAE: 0.2094
Batch Loss: 0.1886, Batch MAE: 0.2422
Batch Loss: 0.2072, Batch MAE: 0.3044
Batch Loss: 0.2134, Batch MAE: 0.3114
Batch Loss: 0.2080, Batch MAE: 0.2909
Batch Loss: 0.1991, Batch MAE: 0.2576
Batch Loss: 0.1831, Batch MAE: 0.2287
Batch Loss: 0.1867, Batch MAE: 0.2243
Epoch 2/20
Batch Loss: 0.1937, Batch MAE: 0.1987
Batch Loss: 0.1745, Batch MAE: 0.1654
Batch Loss: 0.1979, Batch MAE: 0.1776
Batch Loss: 0.1832, Batch MAE: 0.1531
Batch Loss: 0.1812, Batch MAE: 0.1551
Batch Loss: 0.1739, Batch MAE: 0.1553
Batch Loss: 0.1717, Batch MAE: 0.1697
Batch Loss: 0.1615, Batch MAE: 0.1765
Batch Loss: 0.1675, Batch MA

MemoryError: Unable to allocate 18.8 GiB for an array with shape (2457, 1027320) and data type float64

In [ ]:
print("Alex")

Alex
